In [1]:
#!pip install pt_core_news_sm

In [2]:
import spacy
import pt_core_news_sm

In [11]:
import spacy

print("Pipeline:", nlp.pipe_names)
doc = nlp("Eu vou viajar este fim de semana.")
token = doc[2]  # 'I'
token.morph  # 'Case=Nom|Number=Sing|Person=1|PronType=Prs'
dictionary = {"ADP":"adposição", "PROPN": "substantivo próprio" ,"PRON": "pronome", "AUX": "verbo auxiliar", "DET":"artigo", "VERB":"verbo", "NOUN":"substantivo", "PUNCT":"pontuação"}
grammer = [dictionary[token.pos_] for token in doc]
print(grammer)

Pipeline: ['tagger', 'parser', 'ner']
['pronome', 'verbo auxiliar', 'verbo', 'artigo', 'substantivo', 'adposição', 'substantivo', 'pontuação']


In [12]:
with open('morphology_terms.txt', encoding='utf-8') as file:
    arq = file.readlines()

translate_morp = {}
for line in arq:
    l = line.strip().split('\t')
    value = l.pop(0)
    translate_morp[value] = l

translate_morp   

{'VERB': ['verbo'],
 'Nom': ['nome'],
 'Ind': ['indicativo', 'indefinido'],
 'Number': ['número'],
 'Sing': ['singular'],
 'Plur': ['plural'],
 'Person': ['pessoa'],
 '1': ['primeira'],
 '2': ['segunda'],
 '3': ['terceira'],
 'Tense': ['tempo'],
 'Pres': ['presente'],
 'Imp': ['imperativo'],
 'PRON': ['pronome'],
 'PROPN': ['substantivo próprio'],
 'Case': ['caso'],
 'Gender': ['gênero'],
 'Fem': ['feminino'],
 'Masc': ['masculino'],
 'PronType': ['tipo de pronome'],
 'Prs': ['pessoal'],
 'DET': ['determinador'],
 'Definite': ['definido'],
 'Def': ['definido'],
 'Art': ['artigo'],
 'AUX': ['auxiliar'],
 'Mood': ['modo'],
 'VerbForm': ['forma do verbo'],
 'Fin': ['finitivo'],
 'Inf': ['infinitivo'],
 'Ger': ['gerúndio'],
 'Dem': ['demonstrativo'],
 'ADJ': ['adjetivo'],
 'ADV': ['advérbio'],
 'NOUN': ['substantivo'],
 'SCONJ': ['conjunção subordinada'],
 'Sub': ['subjuntivo'],
 'INTJ': ['interjeição'],
 'NUM': ['numeral'],
 'past': ['passado']}

#### Morphology Tool Library Spacy

In [13]:
t = nlp(u"rede")[0]
result = {}

if t.tag_ != "PUNCT":
    try:
        classify = t.tag_.split("|")
        category = classify[0].split("__") 
        classify[0] = category.pop(1)
        result = {"categoria": translate_morp[category[0]][0]}

        for c in classify:
            tag, value = c.split("=")
            if value != "Ind" or result["categoria"] == "verbo":
                result[translate_morp[tag][0]] = translate_morp[value][0] 
            else:
                result[translate_morp[tag][0]] = translate_morp[value][1]
    except:
        result["categoria"] = translate_morp[t.tag_][0]

print(result)

{'categoria': 'substantivo', 'gênero': 'feminino', 'número': 'singular'}


In [14]:
spacy.explain("PROPN")

'proper noun'

In [ ]:
#Criar um arquivo de tradução dos termos morfológicos para o Português

In [15]:
with open('sintaxe_terms.txt', encoding='utf-8') as file:
    arq = file.readlines()

translate_dep = {}
for line in arq:
    value, l = line.strip().split("\t")
    translate_dep[value] = l

In [16]:
for i, _ in enumerate(doc):
    print(doc[i].dep_, doc[i].pos_)
    doc[i].dep_ = translate_dep[doc[i].dep_].upper()
    #doc[i].pos_ =  translate_dep[doc[i].pos_]

nsubj PRON
aux AUX
ROOT VERB
det DET
obj NOUN
compound ADP
compound NOUN
punct PUNCT


In [17]:
from spacy import displacy
from pathlib import Path

svg = displacy.render(doc, style = 'dep', jupyter = False)

In [18]:
displacy.render(doc, style = 'dep', jupyter = True)

In [19]:
for token in doc:
    print(f"{token.head.text} --> {token.dep_.upper()} --> {token.text}")

viajar --> SUJEITO NOMINAL --> Eu
viajar --> AUXILIAR --> vou
viajar --> RAÍZ --> viajar
fim --> DETERMINADOR --> este
viajar --> OBJETO --> fim
fim --> COMPOSTO --> de
fim --> COMPOSTO --> semana
viajar --> PONTUAÇÃO --> .


#### API Sintaxe Tool Return - Json File

In [20]:
from collections import defaultdict

return_ = defaultdict(list)

for token in doc:
    if token.dep_.upper() != "PONTUAÇÃO":
        return_[token.head.text].append((token.dep_, token.text))

return_

defaultdict(list,
            {'viajar': [('SUJEITO NOMINAL', 'Eu'),
              ('AUXILIAR', 'vou'),
              ('RAÍZ', 'viajar'),
              ('OBJETO', 'fim')],
             'fim': [('DETERMINADOR', 'este'),
              ('COMPOSTO', 'de'),
              ('COMPOSTO', 'semana')]})

In [21]:
#!pip install --user svglib

#### API Sintaxe Tool Return - SVG Image 

In [22]:
output_path = Path("svg_file.svg")
output_path.open("w", encoding="utf-8").write(svg)

from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM

drawing = svg2rlg('svg_file.svg')
renderPM.drawToFile(drawing, 'ice.png', fmt='PNG')